In [1]:
from __future__ import print_function

In [2]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import time
import datetime
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

train = pd.read_csv("EURUSD-Tableau-1.csv",  delimiter=';')

## Analyse de la structure du fichier

In [4]:
train.head(10)

,Time,Currency Pair,Amount in Units of 1 million,Normalised BIS Volume (per second),Amount/Capacity Ratio,"Log 1,9",Size Factor,NCFX Mid,Prior Minute Vol,Mid x Min Vol,Norm. Vol,Vol Adjusted Base Spread,Addition for Size,Expected Spread
0,1:00,EURUSD,1,"4,2","0,24","-2,24","0,0","1,17523","0,000022","0,00003","0,000055","0,00006","0,6","1,0"
1,1:00,EURUSD,5,"4,2","1,19","0,27","0,0","1,17523","0,000022","0,00003","0,000055","0,00006","0,6","1,0"
2,1:00,EURUSD,10,"4,2","2,38","1,35","1,4","1,17523","0,000022","0,00003","0,000055","0,00006","1,9","1,9"
3,1:00,EURUSD,15,"4,2","3,57","1,98","2,0","1,17523","0,000022","0,00003","0,000055","0,00006","2,5","2,5"
4,1:00,EURUSD,20,"4,2","4,76","2,43","2,4","1,17523","0,000022","0,00003","0,000055","0,00006","3,0","3,0"
5,1:00,EURUSD,30,"4,2","7,14","3,06","3,1","1,17523","0,000022","0,00003","0,000055","0,00006","3,6","3,6"
6,1:00,EURUSD,50,"4,2","11,90","3,86","3,9","1,17523","0,000022","0,00003","0,000055","0,00006","4,4","4,4"
7,1:00,EURUSD,100,"4,2","23,81","4,94","4,9","1,17523","0,000022","0,00003","0,000055","0,00006","5,5","5,5"
8,1:00,EURUSD,200,"4,2","47,62","6,02","6,0","1,17523","0,000022","0,00003","0,000055","0,00006","6,6","6,6"
9,3:00,EURUSD,1,"3,4","0,29","-1,91","0,0","1,17615","0,000013","0,00002","0,000055","0,00006","0,6","1,0"


In [5]:
train.columns.tolist()

['Time ',
 'Currency Pair',
 'Amount in Units of 1 million',
 'Normalised BIS Volume (per second)',
 'Amount/Capacity Ratio',
 'Log 1,9',
 'Size Factor',
 'NCFX Mid',
 'Prior Minute Vol',
 'Mid x Min Vol',
 'Norm. Vol',
 'Vol Adjusted Base Spread',
 'Addition for Size',
 'Expected Spread']

Le model comprend 14 colonnes, principalement des valeurs numériques et une information temporelle. Nous allons commencer par parcourir ces colonnes une à une, en y appliquant les éventuelles modifications permettant une meilleur exploitation de la donnée.

### -- Time --

In [6]:
train['Time '].sample(10)

111    17:00
147    22:00
11      3:00
123    18:00
145    22:00
17      3:00
143    20:00
46      8:00
90     15:00
39      7:00
Name: Time , dtype: object

Nous avons ici des heures de la journée, plusieurs points à noter :
- stockées en chaîne de caractère donc à convertir 
- Une heure de la journée peut être, répétée sur plusieurs lignes. Chaque ligne correspondant à une valeur distincte de "Amount in Units of 1 million". Nous verrons plus loin si l'on peut s'affranchir de ces "doublons".
- La fréquence peut variée au sein de la série, durant les premières et les dernières heures de la journée, il y à un enregistrement toutes les 2 heures, puis à partir de 7h, il n'y à plus qu'une heure entre chaque enregistrement. Nous allons voir par la suite que ces variations peuvent poser problème pour l'alimentation d'un modèle de Deep Learning.

In [7]:
train['Time '] = pd.to_datetime(train['Time '],format= '%H:%M' ).dt.time

### -- Currency Pair --

In [8]:
train['Currency Pair'].unique()

array(['EURUSD'], dtype=object)

Cette colonne de type String contient le type de monnaie échangée, dans ce fichier de test nous n'avons que la valeur 'EURUSD' (€ --> $) mais nous pouvons importer les différents onglets du fichier source pour avoir toutes les monnaies.

Ceci ne fausse pas l'analyse à ce stade.

### -- Amount in Units of 1 million --

In [9]:
train['Amount in Units of 1 million'].describe()

count    153.000000
mean      47.862745
std       61.239613
min        1.000000
25%       10.000000
50%       20.000000
75%       50.000000
max      200.000000
Name: Amount in Units of 1 million, dtype: float64

Cette colonne contient le montant de transaction en millions de'€ . 5 montants différents sont répétés pour chaque heure.

Format : OK déjà en format numérique
Origine : Valeur saisie
Observation : Nous allons voir si ces lignes ne peuvent pas être calculées à partir des valeurs fournies pour 1M€

### -- Normalised BIS Volume (per second) --

In [10]:
train['Normalised BIS Volume (per second)'].sample(10)

33      4,2
151     1,1
7       4,2
103    11,5
132     6,1
3       4,2
134     6,1
76      8,4
125     6,1
104    11,5
Name: Normalised BIS Volume (per second), dtype: object

Format : KO --> le séparateur virgule entraîne une conversion en chaîne de caractère au chargement, il faut donc procéder à une conversion

Origine : Valeur saisie

Valeurs manquantes : 1 , s'agissant d'une valeur clé nous ne pouvons garder la ligne concernée. Ceci d'autant plus qu'après vérification il s'agit d'un réel doublon.

Observation : le problème de format constaté s'applique à toutes les colonnes suivantes, pour gagner du temps nous pouvons procéder à une correction globale

In [11]:
train.dropna(inplace = True)

columns_to_convert_float = ['Normalised BIS Volume (per second)', 'Amount/Capacity Ratio',
       'Log 1,9', 'Size Factor', 'NCFX Mid','Prior Minute Vol',
       'Mid x Min Vol', 'Norm. Vol', 'Vol Adjusted Base Spread',
       'Addition for Size', 'Expected Spread']

for column in columns_to_convert_float:
        train[column] = train[column].apply(lambda x: float(x.replace(",",".")) )
        print( column + ' : ' + str(type(train[column][0])))



Normalised BIS Volume (per second) : <class 'numpy.float64'>
Amount/Capacity Ratio : <class 'numpy.float64'>
Log 1,9 : <class 'numpy.float64'>
Size Factor : <class 'numpy.float64'>
NCFX Mid : <class 'numpy.float64'>
Prior Minute Vol : <class 'numpy.float64'>
Mid x Min Vol : <class 'numpy.float64'>
Norm. Vol : <class 'numpy.float64'>
Vol Adjusted Base Spread : <class 'numpy.float64'>
Addition for Size : <class 'numpy.float64'>
Expected Spread : <class 'numpy.float64'>


Toutes les colonnes suivantes sont désormais en format numérique.

### -- Amount/Capacity Ratio --

In [12]:
train['Amount/Capacity Ratio'].describe()

count    152.000000
mean       9.884803
std       19.510543
min        0.070000
25%        1.190000
50%        3.500000
75%       10.515000
max      181.820000
Name: Amount/Capacity Ratio, dtype: float64

Format : OK 

Origine : Valeur calculée dans le modèle Excel

Valeurs manquantes : aucune

Observation : le problème de format constaté s'applique à toutes les colonnes suivantes, pour gagner du temps nous pouvons procéder à une correction globale

In [13]:
train['ac_ratio']= train['Amount in Units of 1 million']/train['Normalised BIS Volume (per second)']
train.iloc[0:10, [4,14]]

,Amount/Capacity Ratio,ac_ratio
0,0.24,0.238095
1,1.19,1.190476
2,2.38,2.380952
3,3.57,3.571429
4,4.76,4.761905
5,7.14,7.142857
6,11.90,11.904762
7,23.81,23.809524
8,47.62,47.619048
9,0.29,0.294118


### -- Log 1,9 --

In [14]:
train['Log 1,9'].describe()

count    152.000000
mean       1.805921
std        2.555559
min       -4.160000
25%        0.270000
50%        1.950000
75%        3.660000
max        8.110000
Name: Log 1,9, dtype: float64

Format : OK 

Origine : Valeur calculée dans le modèle Excel

Valeurs manquantes : aucune

Observation : Tout comme pour la valeur précédente, les valeurs calculées dans Excel peuvent l'être directement dans le DataFrame. Nous allons donc alimenter une nouvelle colonne dynamique, comparer le résultat avec les valeurs fournies, et si le test est concluant supprimer la colonne d'origine 

In [15]:
train['LOG']=train.apply(lambda x: math.log(x[14],1.9), axis=1)
train.iloc[0:10, [5,15]]

,"Log 1,9",LOG
0,-2.24,-2.235843
1,0.27,0.271640
2,1.35,1.351555
3,1.98,1.983264
4,2.43,2.431469
5,3.06,3.063178
6,3.86,3.859038
7,4.94,4.938952
8,6.02,6.018866
9,-1.91,-1.906626


### -- Size Factor --

In [16]:
train['Size_Factor'] = train.apply(lambda x: x[15] if x[15]>1 else  0, axis=1)
train.iloc[0:10, [6,16]]

,Size Factor,Size_Factor
0,0.0,0.000000
1,0.0,0.000000
2,1.4,1.351555
3,2.0,1.983264
4,2.4,2.431469
5,3.1,3.063178
6,3.9,3.859038
7,4.9,4.938952
8,6.0,6.018866
9,0.0,0.000000


### -- NCFX Mid and Prior Minute Vol --

Les 2 colonnes suivantes sont des valeurs saisies et sont déjà dans le format attendu. Nous ne nous y attardons pas à ce stade de l'analyse, mais nous pouvons d'ores et déjà imaginer qu'elles feront partir des valeurs clé à prédire, afin d'estimer le Spread.

### -- Mid x min vol --

Format : OK 

Origine : Valeur calculée dans le modèle Excel

Valeurs manquantes : aucune

Observation : la formule est la suivante ('NCFX Mid' * (1 + 'prio minute vol') - 'NCFX Mid')

In [17]:
train['midxmin'] = train['NCFX Mid']*(1 + train['Prior Minute Vol'])-train['NCFX Mid']

### -- Vol adjusted Base spread --

Format : OK 

Origine : Valeur calculée dans le modèle Excel

Valeurs manquantes : aucune

Observation : Il s'agit ici de la valeur maximum entre la colonne 'Mid x min vol' et la colonne 'Norm. Vol'

In [18]:
train["vol_adj_base_spread"] = train[["midxmin", "Norm. Vol"]].max(axis=1)
train.drop(['Vol Adjusted Base Spread'], axis=1)
train.iloc[0:10, [11,18]]

,Vol Adjusted Base Spread,vol_adj_base_spread
0,0.00006,0.000055
1,0.00006,0.000055
2,0.00006,0.000055
3,0.00006,0.000055
4,0.00006,0.000055
5,0.00006,0.000055
6,0.00006,0.000055
7,0.00006,0.000055
8,0.00006,0.000055
9,0.00006,0.000055


### -- Addition for size --

Format : OK 

Origine : Valeur calculée dans le modèle Excel

Valeurs manquantes : aucune

Observation : la formule est la suivante : vol_adj_base_spread * 10000 + siz_factor

In [19]:
train['add_for_size']= train["vol_adj_base_spread"] * 10000 + train['Size_Factor']
train.drop(['Addition for Size'], axis=1)
train.iloc[0:10, [12,19]]

,Addition for Size,add_for_size
0,0.6,0.550000
1,0.6,0.550000
2,1.9,1.901555
3,2.5,2.533264
4,3.0,2.981469
5,3.6,3.613178
6,4.4,4.409038
7,5.5,5.488952
8,6.6,6.568866
9,0.6,0.550000


### -- Expected spread --

Format : OK 

Origine : Valeur calculée dans le modèle Excel

Valeurs manquantes : aucune

Observation : la formule est la suivante : Si add_for_size > 1 alors add_for_size SINON 1

In [20]:
train['exp_spread']=train.apply(lambda x: x[19] if x[19]>1 else  1, axis=1)
train.drop(['Expected Spread'], axis=1)
train.iloc[0:10, [13,20]]

,Expected Spread,exp_spread
0,1.0,1.000000
1,1.0,1.000000
2,1.9,1.901555
3,2.5,2.533264
4,3.0,2.981469
5,3.6,3.613178
6,4.4,4.409038
7,5.5,5.488952
8,6.6,6.568866
9,1.0,1.000000


Nous voilà au bout de l'analyse des données et des manipulations.

Nous allons maintenant supprimer toutes les informations du dataset qui ne nous sont pas indispensables, ou plutôt, que nous pouvons calculer nous même.

In [21]:
columns_to_drop = ['Amount/Capacity Ratio',
       'Log 1,9', 'Size Factor', 
       'Mid x Min Vol','Vol Adjusted Base Spread',
       'Addition for Size', 'Expected Spread']
train = train.drop(columns_to_drop, axis=1)

train.head(10)

,Time,Currency Pair,Amount in Units of 1 million,Normalised BIS Volume (per second),NCFX Mid,Prior Minute Vol,Norm. Vol,ac_ratio,LOG,Size_Factor,midxmin,vol_adj_base_spread,add_for_size,exp_spread
0,01:00:00,EURUSD,1,4.2,1.17523,0.000022,0.000055,0.238095,-2.235843,0.000000,0.000026,0.000055,0.550000,1.000000
1,01:00:00,EURUSD,5,4.2,1.17523,0.000022,0.000055,1.190476,0.271640,0.000000,0.000026,0.000055,0.550000,1.000000
2,01:00:00,EURUSD,10,4.2,1.17523,0.000022,0.000055,2.380952,1.351555,1.351555,0.000026,0.000055,1.901555,1.901555
3,01:00:00,EURUSD,15,4.2,1.17523,0.000022,0.000055,3.571429,1.983264,1.983264,0.000026,0.000055,2.533264,2.533264
4,01:00:00,EURUSD,20,4.2,1.17523,0.000022,0.000055,4.761905,2.431469,2.431469,0.000026,0.000055,2.981469,2.981469
5,01:00:00,EURUSD,30,4.2,1.17523,0.000022,0.000055,7.142857,3.063178,3.063178,0.000026,0.000055,3.613178,3.613178
6,01:00:00,EURUSD,50,4.2,1.17523,0.000022,0.000055,11.904762,3.859038,3.859038,0.000026,0.000055,4.409038,4.409038
7,01:00:00,EURUSD,100,4.2,1.17523,0.000022,0.000055,23.809524,4.938952,4.938952,0.000026,0.000055,5.488952,5.488952
8,01:00:00,EURUSD,200,4.2,1.17523,0.000022,0.000055,47.619048,6.018866,6.018866,0.000026,0.000055,6.568866,6.568866
9,03:00:00,EURUSD,1,3.4,1.17615,0.000013,0.000055,0.294118,-1.906626,0.000000,0.000015,0.000055,0.550000,1.000000


Nous n'allons garder également que les valeurs pour 1M qui est la granularité maximum, les autres montants pouvant être calculés.

## Implémentation :

Nous avons maintenant clairement identifier les Inputs et le Outputs du modèle de donnés.

Les valeurs d'inputs sont :
- Time
- Currency Pair
- Amount in Units of 1 million
- Normalized BIS Volume (per second)
- NCFX Mid
- Prior Minute Vol
- Normal Vol

En retour le modèle aura comme outputs :
- ac_ratio
- LOG
- Size_Factor
- midxmin
- vol_adj_base_spread
- add_for_size
- exp_spread

In [22]:
input_list = ['Time ', 'Currency Pair',
       'Normalised BIS Volume (per second)', 'NCFX Mid', 'Prior Minute Vol',
       'Norm. Vol']

output_list =['ac_ratio', 'LOG', 'Size_Factor', 'midxmin',
       'vol_adj_base_spread', 'add_for_size', 'exp_spread']

input_df = train.iloc[:, 0:7]

input_df.to_csv('input_matrix_ncfx.csv')

Le format de fichier attendu pour alimenter un modèle de Deep Learning est généré en annexe de ce NotebooK.

Et ci-dessous un petit outil de simulation.

In [23]:

df = pd.DataFrame()


def estimate_spread(Hour, currpair, amount) :
    
    matrix = pd.read_csv('input_matrix_ncfx.csv')
    matrix =  matrix.loc[matrix['Amount in Units of 1 million']==1]
    #matrix['Time ']=str(matrix['Time '])
    
    #d = {'Hour': [Hour], 'currpair': [currpair], 'amount' : [amount]}
    df['Hour'] = [Hour]
    df['currpair'] = [currpair]
    df['amount'] = [amount]
    
    df['BISvolume'] = matrix.apply(lambda x : matrix.loc[matrix['Time ']==Hour]['Normalised BIS Volume (per second)'], axis = 1)
    df['NCFXmid'] = matrix.apply(lambda x : matrix.loc[matrix['Time ']==Hour]['NCFX Mid'], axis = 1)
    df['Priormin'] = matrix.apply(lambda x : matrix.loc[matrix['Time ']==Hour]['Prior Minute Vol'], axis = 1)
    df['Normvol'] = matrix.apply(lambda x : matrix.loc[matrix['Time ']==Hour]['Norm. Vol'], axis = 1)
    #Now we add the calculated values
    df['acratio'] = df.apply(lambda x : x[2]/x[3],axis=1)
    df['log']=df.apply(lambda x: math.log(x[7],1.9), axis=1)
    df['sizefactor'] = df.apply(lambda x: x[8] if x[8]>1 else  0, axis=1)
    df['midxmin'] = df['NCFXmid']*(1 + df['Priormin'])-df['NCFXmid']
    df["vol_adj_base_spread"] = df[["midxmin", "Normvol"]].max(axis=1)
    df['add_for_size']= df["vol_adj_base_spread"] * 10000 + df['sizefactor']
    df['exp_spread']=df.apply(lambda x: x[12] if x[12]>1 else  1, axis=1)
    

    return(df)



In [24]:
x = interact(estimate_spread, Hour=['01:00:00','03:00:00', '05:00:00', '06:00:00', '07:00:00',
       '08:00:00', '09:00:00', '10:00:00', '12:00:00', '14:00:00',
       '15:00:00', '16:00:00', '17:00:00', '18:00:00', '19:00:00',
       '20:00:00', '22:00:00'], currpair=['EURUSD','USDGBP','EURGBP'], amount=125)
display(x)

interactive(children=(Dropdown(description='Hour', options=('01:00:00', '03:00:00', '05:00:00', '06:00:00', '0…

<function __main__.estimate_spread(Hour, currpair, amount)>

*NB : Il n'y à que la matrice EURUSD dans ce Notebook